In [ ]:
import pandas as pd
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_bounds
from shapely import force_2d
from shapely.geometry import box, Point, Polygon, MultiPolygon
from scipy.spatial import cKDTree
from sklearn.gaussian_process import GaussianProcessRegressor
from pykrige.rk import Krige
from pykrige.ok import OrdinaryKriging
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
from flood_density.plots import plot_gdf, city_bounds_and_density_plot
from flood_density.preprocess import convert_kml_to_gdf, export_to_geojson, get_bounds_xy_min_max, extract_bounds_polygon, polygon_to_gdf, extract_city_data, coordinates_to_box,points_geocoordinates,convert_points_in_gdf, extract_city_bounds_from_df_to_gdf , gdf_to_geojson, clip_density_to_urban_area

from typing import List, Tuple, Callable

CRS_4326 = 4326


ImportError: cannot import name 'polygon_to_geodataframe' from 'flood_density.preprocess' (/home/andy/projects/flood-insurance-lead-detection/flood_density/preprocess.py)

# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [ ]:
gdf_from_kml = convert_kml_to_gdf('laplata_cascourbano.kml')

In [ ]:
gdf_peligrosidad = export_to_geojson(gdf_from_kml, 'laplata_cascourbano.geojson')

In [ ]:
plot_peligrosidad = plot_gdf(gdf_peligrosidad)

# Etapa 2: Obtener las coordenadas de los puntos de la ciudad, formamos un polígono y luego, transformamos a un gdf.


In [ ]:
#¿Cómo obtengo bounds_dict? son coordenadas de la ciudad de La Plata Desde el archivo geojson

In [ ]:
#Extraemos las coordenadas geográficas de la ciudad 
city_bounds_coordinates= get_bounds_xy_min_max(gdf_peligrosidad)
city_bounds_coordinates

In [ ]:
# Convertimos las coordenadas de la ciudad a un box
boxx = coordinates_to_box(city_bounds_coordinates)

In [ ]:
#Usar el dataframe del polígono que forma las coordenadas de la LP
gdf_la_plata_from_polygon = gpd.GeoDataFrame({'geometry': [boxx]}, crs='EPSG:4326')

In [ ]:
# Convertir a EPSG 32721 (UTM zona 21S)
gdf_la_plata_from_polygon = gdf_la_plata_from_polygon.to_crs(epsg=32721)

# Etapa 3 : Obtener y transformar datos de densidad poblacional de la ciudad de La Plata.

Primero extraemos las filas que pertenecen a la ciudad de La Plata(sigue siendo un .csv), luego convertimos ese dataframe en un geodataframe.

In [ ]:
#Levantamos el dataset completo
df_arg = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv") 

In [ ]:
# Extraemos las filas referidas a la ciudad de La Plata
df_lp_coordinates = extract_city_data(df_arg,'la_plata',city_bounds_coordinates)
df_lp_coordinates

In [ ]:
#Leemos los datos específicos de la ciudad de La Plata
df_lp_coordinates = pd.read_csv('la_plata_population_2020.csv')
df_lp_coordinates


In [ ]:
#Convertir dataframe de los puntos pertenecientes a la ciudad de La Plata en un geodataframe
gdf_lp_coordinates = extract_city_bounds_from_df_to_gdf(df_lp_coordinates, 'Y','X')

In [ ]:
# Extraemos las coordenadas de los puntos dentro de la ciudad de La Plata
casco_urbano = clip_density_to_urban_area(gdf_lp_coordinates,gdf_peligrosidad)

In [ ]:
# Reproyectar a EPSG 32721 (UTM zona 21S)
casco_urbano_utm = casco_urbano.to_crs(epsg=32721)
casco_urbano_utm

In [ ]:
# Calcular centroides en el CRS proyectado
centroides = casco_urbano_utm.geometry.centroid

In [ ]:
# Convertir centroides a un array de coordenadas
coords_centroides = np.array([[pt.x, pt.y] for pt in centroides])

In [ ]:
# Extraer densidades de casco_urbano_utm
values_density_lp = casco_urbano_utm['Z'].values

In [ ]:
#Extraer los límites del polígono de la ciudad de La Plata
bounds_lp = gdf_la_plata_from_polygon.total_bounds  # [minx, miny, maxx, maxy]
bounds_lp

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.scatter(coords_centroides[:, 0], coords_centroides[:, 1],
                c=values_density_lp, s=100,
                cmap='viridis', alpha=0.9,
                edgecolor='black', linewidth=1,
                marker='o')

plt.colorbar(sc, ax=ax, label='Densidad')
ax.set_title("Centroides (EPSG:32721)")
plt.show()

# Etapa 4 : Merge polígono y zona de densidad de la ciudad de La Plata en un plot para corroborar consistencia geográfica.

In [ ]:
city_bounds_and_density_plot(gdf_la_plata_from_polygon, coords_centroides, values_density_lp, bounds_lp)

# Etapa 5 :  Visualización de la densidad poblacional de La Plata usando kriging.

In [ ]:
casco_urbano_utm

In [ ]:
values_density_lp = casco_urbano_utm['Z'].values

In [ ]:
# 1. Preparar los datos (se queda)

def prepare_data(gdf: gpd.GeoDataFrame, value_column: str):
    coords = np.array([[point.x, point.y] for point in gdf.geometry.centroid])
    values = gdf[value_column].values
    print(f"Datos de entrada: {len(coords)}")
    print(f"Rango de valores: {values.min():.2f} - {values.max():.2f}")
    return coords, values

In [ ]:
prepare_data(casco_urbano_utm, 'Z')

In [ ]:
# 2. Ajustar el modelo de Kriging
#def fit_kriging(coords: np.ndarray, values: np.ndarray) -> OrdinaryKriging:
#    OK = OrdinaryKriging(
#        coords[:,0], coords[:,1], values,
#        variogram_model='gaussian',
#        verbose=False,
#        enable_plotting=False
#    )
#    return OK

In [ ]:
#model_fit_kriging = fit_kriging(coords_centroides, values_density_lp)

In [ ]:
# 3. Crear malla para interpolar
#def create_grid(bounds: Tuple[float, float, float, float], step: int = 100) -> Tuple[np.ndarray, np.ndarray]:
#    x = np.linspace(bounds[0], bounds[1], step)
#    y = np.linspace(bounds[2], bounds[3], step)
#    grid_x, grid_y = np.meshgrid(x, y)
#    return grid_x, grid_y

In [ ]:
#grid = create_grid(bounds_lp, step=100)

In [ ]:
##def interpolate_grid(bounds: Tuple[float, float, float, float], step: int = 100) -> Tuple[np.ndarray, np.ndarray]:
#    """
#    Genera las coordenadas de la malla donde luego se predicen valores con OrdinaryKriging.
#    """
#    x = np.linspace(bounds[0], bounds[1], step)
#    y = np.linspace(bounds[2], bounds[3], step)
#    grid_x, grid_y = np.meshgrid(x, y)
#    return grid_x, grid_y


In [ ]:
#grid_x, grid_y = interpolate_grid(bounds_lp, step=100)


In [ ]:
#def predict_grid(model: OrdinaryKriging, grid_x: np.ndarray, grid_y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
#    """
#    Predice valores e incertidumbres en un grid usando OrdinaryKriging.
#    """
#    z, ss = model.execute('grid', grid_x[0], grid_y[:,0])
#    return z, ss


In [ ]:
#predict_grid(model_fit_kriging, grid_x, grid_y)

In [ ]:
#def plot_kriging_results_with_basemap(gdf: gpd.GeoDataFrame, 
#                                      coords: np.ndarray, 
#                                      bounds: Tuple[float, float, float, float], 
#                                      grid_x: np.ndarray, 
#                                      grid_y: np.ndarray, 
#                                      model: OrdinaryKriging, 
#                                      predict_fn: Callable[[OrdinaryKriging, np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]) -> plt.Figure:
#
#    fig, ax = plt.subplots(figsize=(10, 10))
#
#    # Obtener la superficie interpolada desde la función predict_fn
#    grid_z, ss = predict_fn(model, grid_x, grid_y)
#    
#    # Plotear superficie Kriging interpolada
#    contour = ax.contourf(grid_x, grid_y, grid_z, levels=30, cmap='viridis', alpha=0.5)
#    
#
#    # Plotear los polígonos originales
#    casco_urbano_utm.plot(column='Z', 
#               cmap='viridis',
#               alpha=0.5,
#               edgecolor='black',
#               linewidth=1.0,
#               ax=ax)
#
#    # Plotear los polígonos originales con bordes
#    casco_urbano_utm.plot(column='Z', 
#               cmap='viridis',
#               alpha=0.5,
#               edgecolor='black',
#               linewidth=1.0,
#               ax=ax)
#
#    # Plotear los puntos centroides
#    scatter = ax.scatter(coords[:, 0], coords[:, 1],
#                    c=gdf['Z'],
#                    cmap='viridis',
#                    s=30,
#                    edgecolors='black',
#                    linewidths=1,
#                    zorder=5)
#
#    # Ajustar límites del gráfico según los límites de La Plata
#    ax.set_xlim(bounds[0], bounds[2])
#    ax.set_ylim(bounds[1], bounds[3])
#
#    #Mapa base más sutil
#    ctx.add_basemap(ax, crs=casco_urbano_utm.crs.to_string(),
#                   source=ctx.providers.CartoDB.Positron,
#                   alpha = 0.9)
#
#    # Agregar colorbar para el scatter
#    plt.colorbar(scatter, ax=ax, label='Densidad poblacional',shrink=0.7, aspect=25)
#
#    # Agregar título y etiquetas
#    ax.set_title('Interpolación Kriging - Densidad Poblacional La Plata')
#    ax.set_xlabel('X (UTM)')
#    ax.set_ylabel('Y (UTM)')
#
#    plt.tight_layout()
#    plt.show()
#

In [ ]:
#plot_kriging_results_with_basemap(casco_urbano_utm, coords_centroides, bounds_lp, grid_x, grid_y, model_fit_kriging, predict_grid)

In [ ]:
# 5. Pipeline principal
#def run_kriging_pipeline(geodf, value_column, bounds, step=100):
#    coords, values = prepare_data(geodf, value_column)
#    model = fit_kriging(coords, values)
#    grid_xx, grid_yy, z = interpolate_grid(model, bounds, step)
#    plot_kriging_results(grid_xx, grid_yy, z, coords, values, bounds)

----------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 2. Ajustar el modelo de Kriging

def create_kriging_kernel(constant_value=1.0, length_scale=1000.0, noise_level=0.1,
                         length_scale_bounds=(1e-5, 1e5), noise_level_bounds=(1e-10, 1e3)):
    """
    Crea un kernel para un modelo de Kriging (Gaussian Process).
    
    El kernel resultante tiene la forma: C * RBF + WhiteKernel
    donde C es una constante, RBF es el kernel de función de base radial,
    y WhiteKernel modela el ruido en las observaciones.
    
    Parámetros:
    -----------
    constant_value : float, default=1.0
        Valor inicial del ConstantKernel (amplitud del proceso)
    length_scale : float, default=1000.0
        Valor inicial del RBF kernel (escala de correlación espacial)
    noise_level : float, default=0.1
        Valor inicial del WhiteKernel (nivel de ruido)
    length_scale_bounds : tuple, default=(1e-5, 1e5)
        Límites para optimización del parámetro length_scale del RBF
    noise_level_bounds : tuple, default=(1e-10, 1e3)
        Límites para optimización del parámetro noise_level del WhiteKernel
    
    Returns:
    --------
    kernel : sklearn.gaussian_process.kernels.Kernel
        Objeto kernel listo para usar en GaussianProcessRegressor

    """
    # Kernel constante (amplitud)
    constant_kernel = ConstantKernel(constant_value, constant_value_bounds="fixed")
    
    # Kernel RBF (correlación espacial)
    rbf_kernel = RBF(length_scale=length_scale, length_scale_bounds=length_scale_bounds)
    
    # Kernel de ruido
    noise_kernel = WhiteKernel(noise_level=noise_level, noise_level_bounds=noise_level_bounds)
    
    # Combinar kernels: (Constante * RBF) + Ruido
    kernel = constant_kernel * rbf_kernel + noise_kernel
    
    return kernel

In [ ]:
kernel = create_kriging_kernel()
print(kernel)


In [ ]:
# 3. Generar el modelo GaussianProcessRegressor
def gaussian_process_regressor(kernel = kernel, coords: np.ndarray, values: np.ndarray) -> GaussianProcessRegressor:
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, n_restarts_optimizer=10)
    gpr.fit(coords, values)
    return gpr

In [ ]:
# 4. Crear malla para interpolar

#def create_grid(bounds: Tuple[float, float, float, float], step: int = 100) -> Tuple[np.ndarray, np.ndarray]:
#    x = np.linspace(bounds[0], bounds[1], step)
#    y = np.linspace(bounds[2], bounds[3], step)
#    grid_x, grid_y = np.meshgrid(x, y)
#    return grid_x, grid_y


def create_grid(bounds: Tuple[float, float, float, float], step: int = 100) -> Tuple[np.ndarray, np.ndarray]:
    grid_x, grid_y = np.mgrid[bounds[0]:bounds[2]:100j, bounds[1]:bounds[3]:100j]
    grid_coords = np.column_stack([grid_x.ravel(), grid_y.ravel()])
    return grid_x, grid_y, grid_coords

In [ ]:
# Visualización de la densidad poblacional de La Plata usando kriging

# Extraer coordenadas y valores desde el GeoDataFrame
coords =np.array([[point.x, point.y] for point in casco_utm.geometry.centroid])
values = casco_urbano['densidad_poblacional_lp'].values

print(f"Puntos de datos: {len(coords)}")
print(f"Rango de valores: {values.min():.2f} - {values.max():.2f}")

#Definir kernel para el modelo de kriging
kernel = (ConstantKernel(1.0) * RBF(length_scale=1000.0) + 
          WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e3)))

# Crear modelo de kriging
gpr = GaussianProcessRegressor(kernel=kernel,
                                n_restarts_optimizer=10,
                                alpha=1e-04,
                                normalize_y=True)

# Crear grid de puntos para interpolación
bounds_casco_lp = casco_utm.total_bounds
grid_x, grid_y = np.mgrid[bounds_casco_lp[0]:bounds_casco_lp[2]:100j, bounds_casco_lp[1]:bounds_casco_lp[3]:100j]
grid_coords = np.column_stack([grid_x.ravel(), grid_y.ravel()])



# Entrenar el modelo
gpr.fit(coords, values)

#Predecir en el grid
grid_2_kriging, pred_std = gpr.predict(grid_coords, return_std=True)
grid_z = grid_2_kriging.reshape(grid_x.shape)
grid_uncertainty = pred_std.reshape(grid_x.shape)

# Visualización de la superficie Kriging interpolada
fig, ax = plt.subplots(figsize=(8, 10))

# Plotear superficie Kriging interpolada
contour = ax.contourf(grid_x, grid_y, grid_z, levels=30, cmap='viridis', alpha=0.7)

# Plotear los polígonos originales con bordes
casco_utm.plot(column='densidad_poblacional_lp', 
               cmap='viridis',
               alpha=0.5,
               edgecolor='black',
               linewidth=1.0,
               ax=ax)

# Plotear los puntos centroides
scatter = ax.scatter(coords[:, 0], coords[:, 1],
                    c=values,
                    cmap='viridis',
                    s=30,
                    edgecolors='black',
                    linewidths=1,
                    zorder=5)

#Mapa base más sutil
ctx.add_basemap(ax, crs=casco_utm.crs.to_string(),
               source=ctx.providers.CartoDB.Positron,
               alpha = 0.9)

# Agregar colorbar para el scatter
plt.colorbar(scatter, ax=ax, label='Densidad poblacional',shrink=0.7, aspect=25)

# Agregar título y etiquetas
ax.set_title('Interpolación Kriging - Densidad Poblacional La Plata')
ax.set_xlabel('X (UTM)')
ax.set_ylabel('Y (UTM)')

plt.tight_layout()
plt.show()


In [ ]:
def export_to_geotiff(grid_data: np.ndarray, gdf: gpd.GeoDataFrame,  crs_epsg: int = 32721):

     # Crear un GeoDataFrame con los límites del área de La Plata en EPSG:4326
    boxx = coordinates_to_box(city_bounds_coordinates) 
    gdf_la_plata_from_polygon = gpd.GeoDataFrame({'geometry': [boxx]}, crs='EPSG:4326')
    
    # Transformar al CRS del plot
    gdf_la_plata_from_polygon = gdf_la_plata_from_polygon.to_crs(epsg=32721)

    # Obtener límites en coordenadas proyectadas
    bounds_proj = gdf_la_plata_from_polygon.total_bounds  # [xmin, ymin, xmax, ymax]
    height, width = grid_data.shape

    # Calcula la transformación georreferenciada
    transform = from_bounds(bounds_proj[0], bounds_proj[1], bounds_proj[2], bounds_proj[3], width, height)

    with rasterio.open(
        filename,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=grid_data.dtype,
        crs=CRS.from_epsg(crs_epsg),
        transform=transform,
        compress='lzw'
    ) as dst:
        dst.write(grid_data, 1)
        dst.set_band_description(1, 'Densidad Poblacional Kriging')



In [ ]:
# Exportar superficie interpolada
export_to_geotiff(grid_z, gdf_la_plata_from_polygon, 32721)

# Etapa 5 : superposición de mapas de densidad y peligrosidad de inundación de la ciudad de La Plata (QGIS).